In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import json

#import geopandas as gpd

import pgeocode
from geopy.geocoders import Nominatim

import folium
import folium.plugins as plugins

from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, HeatMapWithTime

In [2]:
data = json.loads(open("data/account.json").read())
location = json.loads(open("data/location_history.json").read())
areas_visited = location['Areas you may have visited in the last two years']

location_list = list()

In [3]:
for j in range(len(areas_visited)):
    location_list.append([areas_visited[j]['Time'], areas_visited[j]['City'], areas_visited[j]['Region'], areas_visited[j]['Postal Code']])

In [4]:
location_df = pd.DataFrame (location_list, columns = ['Time', 'City','Region', 'Postal Code'])

In [5]:
location_df

,Time,City,Region,Postal Code
0,2021/02/14 16:16:02 UTC,galloway,Ohio,43119
1,2021/02/12 03:38:56 UTC,galloway,Ohio,48104
2,2021/01/31 15:13:16 UTC,galloway,Ohio,48197
3,2021/01/31 04:01:26 UTC,galloway,Ohio,48212
4,2020/12/24 20:38:21 UTC,galloway,Ohio,48242
...,...,...,...,...
113,2020/04/20 04:02:37 UTC,dayton,Ohio,48109
114,2020/03/02 20:30:43 UTC,dayton,Ohio,48212
115,2021/08/13 19:03:31 UTC,battle creek,Michigan,48104
116,2021/05/17 03:16:52 UTC,battle creek,Michigan,48212


In [6]:
location_df['loc'] = location_df['City'] + ', ' + location_df['Region']

In [7]:
geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
location = geolocator.geocode('4550 galloway, Ohio')
print(location)
print((location.latitude, location.longitude))

4550, Galloway Road, Bogart, Perkins Township, Erie County, Ohio, 44870, United States
(41.39761616382542, -82.6461944598625)


In [8]:
def loc_convert_lat(x):
    
    geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
    location = geolocator.geocode(x)
    return location.latitude

def loc_convert_lang(x):
    
    geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
    location = geolocator.geocode(x)
    return location.longitude

def loc_convert_address(x):
    
    geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
    location = geolocator.geocode(x)
    return location

In [9]:
location_df['address'] = location_df['loc'].apply(loc_convert_address)
location_df['lat'] = location_df['loc'].apply(loc_convert_lat)
location_df['long'] = location_df['loc'].apply(loc_convert_lang)
location_df

,Time,City,Region,Postal Code,loc,address,lat,long
0,2021/02/14 16:16:02 UTC,galloway,Ohio,43119,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
1,2021/02/12 03:38:56 UTC,galloway,Ohio,48104,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
2,2021/01/31 15:13:16 UTC,galloway,Ohio,48197,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
3,2021/01/31 04:01:26 UTC,galloway,Ohio,48212,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
4,2020/12/24 20:38:21 UTC,galloway,Ohio,48242,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
...,...,...,...,...,...,...,...,...
113,2020/04/20 04:02:37 UTC,dayton,Ohio,48109,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607
114,2020/03/02 20:30:43 UTC,dayton,Ohio,48212,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607
115,2021/08/13 19:03:31 UTC,battle creek,Michigan,48104,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427
116,2021/05/17 03:16:52 UTC,battle creek,Michigan,48212,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427


In [10]:
#location_df['coordinates'] = list(zip(location_df.lat, location_df.long))

In [11]:
selected_columns = location_df[["Time", "lat", "long"]]
new_df = selected_columns.copy()
new_df

,Time,lat,long
0,2021/02/14 16:16:02 UTC,39.913921,-83.163334
1,2021/02/12 03:38:56 UTC,39.913921,-83.163334
2,2021/01/31 15:13:16 UTC,39.913921,-83.163334
3,2021/01/31 04:01:26 UTC,39.913921,-83.163334
4,2020/12/24 20:38:21 UTC,39.913921,-83.163334
...,...,...,...
113,2020/04/20 04:02:37 UTC,39.758948,-84.191607
114,2020/03/02 20:30:43 UTC,39.758948,-84.191607
115,2021/08/13 19:03:31 UTC,42.319255,-85.182427
116,2021/05/17 03:16:52 UTC,42.319255,-85.182427


In [12]:
lat_long_list = []
for i in new_df['Time'].unique():
    temp=[]
    for index, instance in new_df[new_df['Time'] == i].iterrows():
        temp.append([instance['lat'],instance['long']])
    lat_long_list.append(temp)

In [13]:
#converting it to datetime format
new_df['Time']= pd.to_datetime(new_df['Time'])
#creating a time index
time_index = []
for i in new_df['Time'].unique():
    time_index.append(i)
#formatting the index
date_strings = [d.strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]

In [14]:
#Choosing the map type 
m = folium.Map(location=[42.2808, -83.7430],zoom_start = 5, tiles="openstreetmap",attr="Stadia.AlidadeSmoothDark")
#Plot it on the map
HeatMapWithTime(lat_long_list,radius=10,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=0.7).add_to(m)
# Display the map

# Adds tool to the top right
from folium.plugins import MeasureControl
m.add_child(MeasureControl())

m

In [15]:
from folium.plugins import MarkerCluster

In [16]:
m_2 = folium.Map(location=[42.2808, -83.7430], tiles='openstreetmap', zoom_start=5)

# Add points to the map
for idx, row in location_df.iterrows():
    Marker([row['lat'], row['long']]).add_to(m_2)

# Display the map
#m_2

In [17]:
import math


In [18]:
# Create the map
m_3 = folium.Map(location=[42.2808, -83.7430], tiles='openstreetmap', zoom_start=5)

# Add points to the map
mc = MarkerCluster()
for idx, row in location_df.iterrows():
    if not math.isnan(row['long']) and not math.isnan(row['lat']):
        mc.add_child(Marker([row['lat'], row['long']]))
m_3.add_child(mc)

# Adds tool to the top right
from folium.plugins import MeasureControl
m_3.add_child(MeasureControl())

# Display the map
m_3